In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os,math
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'covtype.csv'
classes = 7
data=pd.read_csv(path, header=None)
X=data.iloc[:,:54]
Y=data.iloc[:,-1]
X = MinMaxScaler().fit_transform(X)
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(581012, 54) (581012, 7)


In [3]:
X_train,X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.483659)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(300000, 54) (300000, 7)
(281012, 54) (281012, 7)


In [6]:
x_train,x_test = X_train,X_test
n = X_train.shape[1]
# convert 2-D to 3-D.
for i in range(1,n):
  model = Sequential()
  model.add(Dense(n,name='feature', activation='relu',input_shape=(n,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=2096,verbose=1)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  x_train = np.concatenate((x_train, features1), axis=1) # concatenate on horizontal axis
  x_test = np.concatenate((x_test, features2), axis=1)
print(x_train.shape,x_test.shape)

144/144 [==============================] - 0s 1ms/step - loss: 1.1546
(300000, 2916) (281012, 2916)


In [7]:
from PIL import Image
xtrain = []
xtest = []
for i in range(len(x_train)):
    img = Image.fromarray(x_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtrain.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtrain = np.array(xtrain,dtype='float32')
for i in range(len(x_test)):
    img = Image.fromarray(x_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtest.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtest = np.array(xtest,dtype='float32')
print(xtrain.shape,xtest.shape)

(300000, 32, 32, 3) (281012, 32, 32, 3)


In [8]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(xtrain,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(xtest,Y_test))

Epoch 1/20
1172/1172 [==============================] - 106s 91ms/step - loss: 0.7233 - accuracy: 0.6893 - val_loss: 1.5521 - val_accuracy: 0.5455
Epoch 2/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.6218 - accuracy: 0.7319 - val_loss: 1.0090 - val_accuracy: 0.6097
Epoch 3/20
1172/1172 [==============================] - 102s 87ms/step - loss: 0.6023 - accuracy: 0.7392 - val_loss: 1.2773 - val_accuracy: 0.5830
Epoch 4/20
1172/1172 [==============================] - 100s 86ms/step - loss: 0.5687 - accuracy: 0.7550 - val_loss: 0.9071 - val_accuracy: 0.6584
Epoch 5/20
1172/1172 [==============================] - 100s 85ms/step - loss: 0.5184 - accuracy: 0.7771 - val_loss: 0.7902 - val_accuracy: 0.7196
Epoch 6/20
1172/1172 [==============================] - 101s 86ms/step - loss: 0.5003 - accuracy: 0.7872 - val_loss: 0.9787 - val_accuracy: 0.6526
Epoch 7/20
1172/1172 [==============================] - 101s 86ms/step - loss: 0.4726 - accuracy: 0.7995 - val_loss: 2.

In [9]:
score = model.evaluate(xtest,Y_test)
acc = model.evaluate(xtrain,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

9375/9375 [==============================] - 88s 9ms/step - loss: 1.9150 - accuracy: 0.4674
training accuracy:  0.4674000144004822
testing accuracy:  0.470186322927475
